In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import gpytorch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float32)

%matplotlib inline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:
# Load the dataset
train_data = np.load("./Dataset/train_no_pos.npy")
val_data = np.load("./Dataset/val_no_pos.npy")
test_data = np.load("./Dataset/test_no_pos.npy")

train_x = torch.from_numpy(train_data[:, :-1]) #.to(device)
# train_x = train_x.view(-1, 18, 1)

train_y = torch.from_numpy(train_data[:, -1]) #.to(device)
# train_y = train_y.view(-1, 1)

print(train_x.shape, train_y.shape)

torch.Size([4397, 18]) torch.Size([4397])


In [4]:
# Exact GPs
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Small MLP Feature Extractor
class MLP(nn.Sequential):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        
        self.add_module("linear1", nn.Linear(input_dim, 256))
        self.add_module("relu1", nn.ReLU())
        self.add_module("b1", nn.BatchNorm1d(256))
        self.add_module("linear2", nn.Linear(256, 128))
        self.add_module("relu2", nn.ReLU())
        self.add_module("linear3", nn.Linear(128, output_dim))
        self.add_module("dropout", nn.Dropout(p=0.25))
    
# MLP
feature_extractor = MLP(input_dim=18, output_dim=2)

# MLP - GP
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2)),
                num_dims=2, grid_size=100
            )
            self.feature_extractor = feature_extractor

            # This module will scale the NN features so that they're nice values
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor(x)
            projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice"

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
# model = ExactGPModel(train_x, train_y, likelihood)
model = GPRegressionModel(train_x, train_y, likelihood)

In [5]:
import time

# Using GPU
model = model.to(device)
likelihood = likelihood.to(device)
train_x = train_x.to(device)
train_y = train_y.to(device)

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # ExactGP

# MLP GP
optimizer = torch.optim.AdamW([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 240
for i in range(training_iter):
    
    start_time = time.time()
    
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    
    # Output from model
    output = model(train_x)
    
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    
    print('Iter %d/%d - Loss: %.3f time: %.3f' % (
        i + 1, training_iter, loss.item(),
        (time.time() - start_time)
    ))
    
    optimizer.step()

Iter 1/240 - Loss: nan time: 0.743
Iter 2/240 - Loss: nan time: 0.059
Iter 3/240 - Loss: 0.687 time: 0.059
Iter 4/240 - Loss: 0.649 time: 0.055
Iter 5/240 - Loss: 0.612 time: 0.055
Iter 6/240 - Loss: 0.574 time: 0.053
Iter 7/240 - Loss: 0.536 time: 0.054
Iter 8/240 - Loss: 0.498 time: 0.057
Iter 9/240 - Loss: 0.458 time: 0.058
Iter 10/240 - Loss: 0.418 time: 0.057
Iter 11/240 - Loss: 0.379 time: 0.056
Iter 12/240 - Loss: 0.338 time: 0.053
Iter 13/240 - Loss: 0.297 time: 0.055
Iter 14/240 - Loss: 0.256 time: 0.057
Iter 15/240 - Loss: 0.215 time: 0.057
Iter 16/240 - Loss: 0.174 time: 0.058
Iter 17/240 - Loss: 0.135 time: 0.057
Iter 18/240 - Loss: 0.092 time: 0.053
Iter 19/240 - Loss: 0.053 time: 0.052
Iter 20/240 - Loss: 0.015 time: 0.052
Iter 21/240 - Loss: -0.025 time: 0.054
Iter 22/240 - Loss: -0.064 time: 0.054
Iter 23/240 - Loss: -0.102 time: 0.053
Iter 24/240 - Loss: -0.140 time: 0.056
Iter 25/240 - Loss: -0.176 time: 0.058
Iter 26/240 - Loss: -0.209 time: 0.057
Iter 27/240 - Loss:

In [6]:
val_x = torch.from_numpy(val_data[:, :-1]).to(device)
# val_x = val_x.view(-1, 18, 1)

val_y = torch.from_numpy(val_data[:, -1]).to(device)
# val_y = val_y.view(-1, 1)

print(val_x.shape, val_y.shape)

torch.Size([489, 18]) torch.Size([489])


In [7]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(val_x))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()

# RMSE
rmse = torch.sqrt(torch.mean(torch.pow(observed_pred.mean - val_y, 2)))
print(f"RMSE: {rmse:.3f}")

RMSE: 0.146


In [8]:
mean = mean.cpu()
lower = lower.cpu()
upper = upper.cpu()

train_x = train_x.cpu()
train_y = train_y.cpu()
val_x = val_x.cpu()
val_y = val_y.cpu()

train_x.shape, train_y.shape, val_x.shape, val_y.shape

(torch.Size([4397, 18]),
 torch.Size([4397]),
 torch.Size([489, 18]),
 torch.Size([489]))

In [10]:
%matplotlib qt
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(18, 12))
    X = np.linspace(0, mean.shape[0], mean.shape[0])*0.05
    # Plot training data as black stars
    ax.plot(X, val_y.numpy())
    # Plot predictive means as blue line
    ax.plot(X, mean.numpy())
    # Shade between the lower and upper confidence bounds
    ax.fill_between(X, lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_xlim([10, 15])
    # ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])